# Import Market Capitalization Prices

In [28]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

## Initialize classes

In [88]:
import os
import sys
import configparser
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from wrangler.modules.assets.etl import cryptoMCExtractor as crypto
from utils.modules.etl.load import noSQLwls as nosql

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    crypto = importlib.reload(crypto)
    nosql = importlib.reload(nosql)

clsNoSQL = nosql.NoSQLWorkLoads(desc="get crypto macket capitalization data")
''' optional - if not specified class will use the default values '''
prop_kwargs = {"WRITE_TO_FILE":True,   # necessary to emulate the etl dag
              }
clsMC = crypto.CryptoMarkets(
    desc='get crypto macket capitalization data', **prop_kwargs)
# print(dir(clsMC))
print("\nClass initialization and load complete!")

All python packages in rezaware loaded successfully!
All assets-module etl-packages in function-CryptoMarket imported successfully!
All etl-module load-packages in function-mongodbwls imported successfully!
mongodbwls Class initialization complete
mongodbwls Class initialization complete
CryptoMarket Class initialization complete

Class initialization and load complete!


## get daily prices

In [89]:
from bson.json_util import dumps

_source = 'coinmarketcap'
_data = clsMC.get_daily_mc_data(data_owner=_source)
print(len(_data))
# print(dumps(_data, indent=2))

coinmarketcap
Total 5000 documents, duccessful insert count = 5000 & modify count = 0
5000


In [87]:
__as_type__ = 'dict'
__database__ = "tip-marketcap"
__collection__ = "coinmarketcap.2022-11-18"
__find__ = {'asset.symbol':
            {"$regex" : 'eth'},
           }
clsNoSQL.connect = {'DBAUTHSOURCE':__database__}
print(clsNoSQL.connect)
# clsNoSQL.connect={}
_data = clsNoSQL.read_documents(
    as_type=__as_type__,
    db_name = __database__,
    db_coll=__collection__,
    doc_find={}
)
print(dumps(_data, indent=2))

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True, authsource='tip-marketcap', authmechanism='SCRAM-SHA-256')
[
  {
    "_id": {
      "$oid": "63772be2ac90cbcf8b1c071a"
    },
    "extract.source.id": 1,
    "extract.source.name": "coinmarketcap",
    "extract.datetime": "2022-11-18T06:53:17.424Z",
    "asset.name": "Bitcoin",
    "asset.symbol": "BTC",
    "asset.supply": 19210431,
    "asset.price": 16789.134913262402,
    "marketcap.value": 322526517800.91833,
    "marketcap.rank": 1,
    "marketcap.updated": "2022-11-18T06:50:00.000Z"
  },
  {
    "_id": {
      "$oid": "63772be2ac90cbcf8b1c071b"
    },
    "extract.source.id": 1027,
    "extract.source.name": "coinmarketcap",
    "extract.datetime": "2022-11-18T06:53:17.424Z",
    "asset.name": "Ethereum",
    "asset.symbol": "ETH",
    "asset.supply": 122373866,
    "asset.price": 1216.8857150920808,
    "marketcap.value": 148915009701.03018,
    "marketcap.rank": 2,
    "marketcap.updated":

In [41]:
import json
from itertools import chain

f = open ('coingeko.json', "r")
_results = json.loads(f.read())
# _extract_dt = _results['status']['timestamp']
# print(_extract_dt)
_mc_dict_list = []
# for _data in _results['data']:
for _data in _results:
    _mc_dict_list.append(
    {
        "extract.source.id":_data['id'],
#         "extract.source.name":data_owner,
        "extract.datetime":_data['last_updated'],
        "asset.name":_data['name'],
        "asset.symbol":_data['symbol'],
        "asset.supply":int(_data['circulating_supply']),
        "asset.price":float(_data['current_price']),
        "marketcap.value":float(_data['market_cap']),
        "marketcap.rank":int(_data['market_cap_rank']),
        "marketcap.updated":_data['last_updated'],
    }
)
print(json.dumps(_mc_dict_list,indent=2))
    
# result = [dict([key, float(value)]
#                for key, value in data.items() if isinstance(value,float))
#           for data in _mc_dict['data']]
# print(result)
# break

[
  {
    "extract.source.id": "bitcoin",
    "extract.datetime": "2022-11-18T05:39:20.777Z",
    "asset.name": "Bitcoin",
    "asset.symbol": "btc",
    "asset.supply": 19210393,
    "asset.price": 16816.06,
    "marketcap.value": 323469629138.0,
    "marketcap.rank": 1,
    "marketcap.updated": "2022-11-18T05:39:20.777Z"
  },
  {
    "extract.source.id": "ethereum",
    "extract.datetime": "2022-11-18T05:39:22.602Z",
    "asset.name": "Ethereum",
    "asset.symbol": "eth",
    "asset.supply": 120514577,
    "asset.price": 1220.75,
    "marketcap.value": 147256245121.0,
    "marketcap.rank": 2,
    "marketcap.updated": "2022-11-18T05:39:22.602Z"
  },
  {
    "extract.source.id": "tether",
    "extract.datetime": "2022-11-18T05:35:33.133Z",
    "asset.name": "Tether",
    "asset.symbol": "usdt",
    "asset.supply": 65944685875,
    "asset.price": 0.999488,
    "marketcap.value": 65983251146.0,
    "marketcap.rank": 3,
    "marketcap.updated": "2022-11-18T05:35:33.133Z"
  },
  {
    "ex